# Verse Interpreter development

Essential installs:
- `pip3 install .....`


This version is used to test the first steps for the verse interpreter. The final version will be used as a full python file.

In [15]:
"a" + "{2}"

'a{2}'

In [22]:
import string
from enum import Enum
from collections import namedtuple
import copy
import itertools

# Error Class

In [23]:
class ErrorType(Enum):
    SyntaxError = 'Wrong Syntax at'
    SemanticError = 'Wrong Semantics at'
    UnkownError = 'Operation Failure'

In [24]:
class NodeStatus(Enum):
    # Data
    VALUE_RECEIVABLE = None
    NOT_ASSIGNED_YET = None
    ERROR = None

# Logger

In [25]:
class Logger:
    def __init__(self):{}
        
    def __log__(self, string:str):{}

    def __log_error__(self,string:str, type:ErrorType):{}

class Console_Logger(Logger):

    def __log__(self, string:str):
        print(string)

    def __log_error__(self,string:str, type:ErrorType):       
        print("ERROR| " + type.value + ": " + string)

## Setting Token Enum for testing purpose

In [26]:
class TokenTypes(Enum):
    # Data
    INTEGER = int
    IDENTIFIER = string #Names/Variables
    INT_TYPE = "int"
    TUPLE_TYPE = "tuple"
    ARRAY_TYPE = "array"
    FAIL = "false?"
    # Aritmetics
    PLUS = "+"
    MINUS = "-"
    MULTIPLY = "*"
    DIVIDE = "/"
    GREATER = ">"
    GREATEREQ = ">="
    LOWER = "<"
    LOWEREQ = "<="
    CHOICE = "|"
    # Mehtods
    FOR = "for"
    DO = "do"
    IF = "if"
    THEN = "then"
    ELSE = "else"
    # Else
    EOF = None
    COLON = ":"
    COMMA=","
    SEMICOLON =";"
    BINDING =":="
    LBRACKET = "("
    RBRACKET = ")"
    SBL = "["
    SBR = "]"
    CBL = "{"
    CBR = "}"
    EQUAL = "="
    SCOPE = ":"
    DOT = "."

In [27]:
list(TokenTypes)

[<TokenTypes.INTEGER: <class 'int'>>,
 <TokenTypes.IDENTIFIER: <module 'string' from 'C:\\Users\\chris\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\string.py'>>,
 <TokenTypes.INT_TYPE: 'int'>,
 <TokenTypes.TUPLE_TYPE: 'tuple'>,
 <TokenTypes.ARRAY_TYPE: 'array'>,
 <TokenTypes.FAIL: 'false?'>,
 <TokenTypes.PLUS: '+'>,
 <TokenTypes.MINUS: '-'>,
 <TokenTypes.MULTIPLY: '*'>,
 <TokenTypes.DIVIDE: '/'>,
 <TokenTypes.GREATER: '>'>,
 <TokenTypes.GREATEREQ: '>='>,
 <TokenTypes.LOWER: '<'>,
 <TokenTypes.LOWEREQ: '<='>,
 <TokenTypes.CHOICE: '|'>,
 <TokenTypes.FOR: 'for'>,
 <TokenTypes.DO: 'do'>,
 <TokenTypes.IF: 'if'>,
 <TokenTypes.THEN: 'then'>,
 <TokenTypes.ELSE: 'else'>,
 <TokenTypes.EOF: None>,
 <TokenTypes.COLON: ':'>,
 <TokenTypes.COMMA: ','>,
 <TokenTypes.SEMICOLON: ';'>,
 <TokenTypes.BINDING: ':='>,
 <TokenTypes.LBRACKET: '('>,
 <TokenTypes.RBRACKET: ')'>,
 <TokenTypes.SBL: '['>,
 <TokenTypes.SBR: ']'>,
 <TokenTypes.CBL: '{'>,
 <TokenTypes.CBR: '}'>,
 <TokenTypes.EQUAL: '='>,
 <Token

In [28]:
class Token:
    def __init__(self, type: TokenTypes, value) -> None:
        self.type = type
        self.value = value
    
    def __info__(self):
         return "{}: {}".format(self.type, self.value)

## Lexer

In [29]:
class lexicon:
    def __init__(self, input: string):
        self.input = input
        self.index = 0
        self.current_char = self.input[self.index]
    
    # moves the pointer a character forward
    def forward(self) -> None:
        self.index += 1

        # checks if index is out of range
        if (self.index >= len(self.input)):
            self.current_char = None
            return
        
        self.current_char = self.input[self.index]
    
    def backward(self) -> None:
        self.index -= 1

        # checks if index is out of range
        if self.index < 0:
            self.current_char = None
            return
        
        self.current_char = self.input[self.index]
    
    def get_int(self) -> int:
        if self.index >= len(self.input):
            return None
        
        result = self.input[self.index]

        # checks if there are multiple digits
        while True:
            self.forward()

            if self.index < len(self.input) and self.input[self.index] != None and self.input[self.index].isnumeric():
                result += self.input[self.index]
            else:
                self.backward()
                break

        return int(result)
    
    def get_var(self) -> string:
        if self.index >= len(self.input):
            return None
        
        result = self.input[self.index]

        # checks if there is a longer variable name
        while True:
            self.forward()

            if self.index < len(self.input) and self.input[self.index] != None and self.input[self.index].isalpha():
                result += self.input[self.index]
            elif self.index < len(self.input) and self.input[self.index] != None and self.input[self.index] == '?':
                result += self.input[self.index]
            else:
                self.backward()
                break
        
        return result
    
    def get_binding(self):
        if self.index >= len(self.input) and self.index + 1 >= len(self.input):
            return None
        
        result = self.input[self.index]
        self.forward()
        
        if self.index < len(self.input) and self.input[self.index] != None:
                result += self.input[self.index]

        match result:
            case TokenTypes.BINDING.value:
                return Token(TokenTypes.BINDING, TokenTypes.BINDING.value)
        
        self.backward()
        return Token(TokenTypes.COLON, TokenTypes.COLON.value)
    
    def get_greater_eq(self):
        if self.index >= len(self.input) and self.index + 1 >= len(self.input):
            return None
        
        result = self.input[self.index]
        self.forward()
        
        if self.index < len(self.input) and self.input[self.index] != None:
                result += self.input[self.index]

        match result:
            case TokenTypes.GREATEREQ.value:
                return Token(TokenTypes.GREATEREQ, TokenTypes.GREATEREQ.value)
        
        self.backward()
        return Token(TokenTypes.GREATER, TokenTypes.GREATER.value)
    
    def get_lower_eq(self):
        if self.index >= len(self.input) and self.index + 1 >= len(self.input):
            return None
        
        result = self.input[self.index]
        self.forward()
        
        if self.index < len(self.input) and self.input[self.index] != None:
                result += self.input[self.index]

        match result:
            case TokenTypes.LOWEREQ.value:
                return Token(TokenTypes.LOWEREQ, TokenTypes.LOWEREQ.value)
        
        self.backward()
        return Token(TokenTypes.LOWER, TokenTypes.LOWER.value)
    
    def get_token(self, char: string) -> Token:
        token = self.check_for_tokentypes(char)

        if token.type != TokenTypes.EOF:
            return token
        
        if char == None:
            return token
            
        # skip spaces.
        if char == ' ':
            self.forward()
            return self.get_token(self.current_char)
        
        if char == TokenTypes.COLON:
            return self.get_next(TokenTypes.BINDING.value)

        # checks if the current character is a number.
        if char.isnumeric():
            result = self.get_int()
            return Token(TokenTypes.INTEGER, result)
        
        if char.isalpha():
            result = self.get_var()
            token = self.check_for_tokentypes(result)
            if(token.type == TokenTypes.EOF):
                token = Token(TokenTypes.IDENTIFIER, result)  
                  
        return token

    def check_for_tokentypes(self, char: string) -> Token:
        # checks if the current character is a supported token type.
        match char:
            case TokenTypes.INTEGER.value:
                return Token(TokenTypes.INTEGER, TokenTypes.INTEGER.value)
            case TokenTypes.IDENTIFIER.value:
                return Token(TokenTypes.IDENTIFIER, TokenTypes.IDENTIFIER.value)
            case TokenTypes.INT_TYPE.value:
                return Token(TokenTypes.INT_TYPE, TokenTypes.INT_TYPE.value)
            case TokenTypes.TUPLE_TYPE.value:
                return Token(TokenTypes.TUPLE_TYPE, TokenTypes.TUPLE_TYPE.value)
            case TokenTypes.ARRAY_TYPE.value:
                return Token(TokenTypes.ARRAY_TYPE, TokenTypes.ARRAY_TYPE.value)
            case TokenTypes.FAIL.value:
                return Token(TokenTypes.FAIL, TokenTypes.FAIL.value)
            case TokenTypes.PLUS.value:
                return Token(TokenTypes.PLUS, TokenTypes.PLUS.value)
            case TokenTypes.MINUS.value:
                return Token(TokenTypes.MINUS, TokenTypes.MINUS.value)
            case TokenTypes.MULTIPLY.value:
                return Token(TokenTypes.MULTIPLY, TokenTypes.MULTIPLY.value)
            case TokenTypes.DIVIDE.value:
                return Token(TokenTypes.DIVIDE, TokenTypes.DIVIDE.value)
            case TokenTypes.GREATER.value:
                return self.get_greater_eq()
            case TokenTypes.GREATEREQ.value:
                return Token(TokenTypes.GREATEREQ, TokenTypes.GREATEREQ.value)
            case TokenTypes.LOWER.value:
                return self.get_lower_eq()
            case TokenTypes.LOWEREQ.value:
                return Token(TokenTypes.LOWEREQ, TokenTypes.LOWEREQ.value)
            case TokenTypes.CHOICE.value:
                return Token(TokenTypes.CHOICE, TokenTypes.CHOICE.value)
            case TokenTypes.FOR.value:
                return Token(TokenTypes.FOR, TokenTypes.FOR.value)
            case TokenTypes.DO.value:
                return Token(TokenTypes.DO, TokenTypes.DO.value)
            case TokenTypes.IF.value:
                return Token(TokenTypes.IF, TokenTypes.IF.value)
            case TokenTypes.THEN.value:
                return Token(TokenTypes.THEN, TokenTypes.THEN.value)
            case TokenTypes.ELSE.value:
                return Token(TokenTypes.ELSE, TokenTypes.ELSE.value)
            case TokenTypes.EOF.value:
                return Token(TokenTypes.EOF, TokenTypes.EOF.value)
            case TokenTypes.COLON.value:
                return self.get_binding()
            case TokenTypes.COMMA.value:
                return Token(TokenTypes.COMMA, TokenTypes.COMMA.value)
            case TokenTypes.SEMICOLON.value:
                return Token(TokenTypes.SEMICOLON, TokenTypes.SEMICOLON.value)
            case TokenTypes.BINDING.value:
                return Token(TokenTypes.BINDING, TokenTypes.BINDING.value)
            case TokenTypes.LBRACKET.value:
                return Token(TokenTypes.LBRACKET, TokenTypes.LBRACKET.value)
            case TokenTypes.RBRACKET.value:
                return Token(TokenTypes.RBRACKET, TokenTypes.RBRACKET.value)
            case TokenTypes.SBL.value:
                return Token(TokenTypes.SBL, TokenTypes.SBL.value)
            case TokenTypes.SBR.value:
                return Token(TokenTypes.SBR, TokenTypes.SBR.value)
            case TokenTypes.CBL.value:
                return Token(TokenTypes.CBL, TokenTypes.CBL.value)
            case TokenTypes.CBR.value:
                return Token(TokenTypes.CBR, TokenTypes.CBR.value)
            case TokenTypes.EQUAL.value:
                return Token(TokenTypes.EQUAL, TokenTypes.EQUAL.value)
            case TokenTypes.SCOPE.value:
                return Token(TokenTypes.SCOPE, TokenTypes.SCOPE.value)
            case TokenTypes.DOT.value:
                return Token(TokenTypes.DOT, TokenTypes.DOT.value)
            case _:
                return Token(TokenTypes.EOF, None)

## Testing Lexer Types

In [30]:
lexer = lexicon("x:4; >=; 1..10")

while lexer.current_char is not None:
    token = lexer.get_token(lexer.current_char)
    print(str(token.value) + " is of the tokentype: " + str(token.type))
    lexer.forward()

x is of the tokentype: TokenTypes.IDENTIFIER
: is of the tokentype: TokenTypes.COLON
4 is of the tokentype: TokenTypes.INTEGER
; is of the tokentype: TokenTypes.SEMICOLON
>= is of the tokentype: TokenTypes.GREATEREQ
; is of the tokentype: TokenTypes.SEMICOLON
1 is of the tokentype: TokenTypes.INTEGER
. is of the tokentype: TokenTypes.DOT
. is of the tokentype: TokenTypes.DOT
10 is of the tokentype: TokenTypes.INTEGER


## ParserNode

In [31]:
#Class that takes a parsed node, containes information if node could have been parsed
class ParsedNode:
    def __init__(self, node, hasSyntaxError:bool ):
        self.node = node
        self.hasSyntaxError = hasSyntaxError

## Syntax Tree Nodes

In [32]:
class BaseNode:
    def __init__(self, token) -> None:
        self.token = token
    
    def visit(self, node):
        if isinstance(node, ProgramNode):
                return self.visit_programNode(node)
        elif isinstance(node, BlockNode):
                return self.visit_blockNode(node)
        elif isinstance(node, ScopeNode):
                return self.visit_scopeNode(node)
        elif isinstance(node, OperatorNode):
                return self.visit_operatorNode(node)
        elif isinstance(node, NumberNode):
                return self.visit_numberNode(node)
        elif isinstance(node, UnaryNode):
                return self.visit_unaryNode(node)
        elif isinstance(node, ParsedNode):
             return self.visit(node.node)
        elif isinstance(node, ForNode):
            return self.visit_forNode(node)
        elif isinstance(node, ScopeNode):
            return self.visit
    
    def visit_programNode(self, node):
        return self.visit(node.node)

    def visit_blockNode(self, node):
        for n in node.nodes:
             return self.visit(n)

    def visit_scopeNode(self, node):
        pass

    def visit_unaryNode(self, node):
        pass
        
    def visit_operatorNode(self, node):
        match node.token.type:
            case TokenTypes.DIVIDE:
                return self.visit(node.leftNode) // self.visit(node.rightNode)
            case TokenTypes.MULTIPLY:
                return self.visit(node.leftNode) * self.visit(node.rightNode)
            case TokenTypes.PLUS:
                return self.visit(node.leftNode) + self.visit(node.rightNode)
            case TokenTypes.MINUS:
                return self.visit(node.leftNode) - self.visit(node.rightNode)   
            case TokenTypes.EQUAL:
                if(self.visit(node.leftNode) == self.visit(node.rightNode)):
                    return self.visit(node.leftNode)
                return ""
            case TokenTypes.GREATER:
                if(self.visit(node.leftNode) > self.visit(node.rightNode)):
                    return self.visit(node.leftNode)
                return ""
            case TokenTypes.LOWER:
                 if(self.visit(node.leftNode) < self.visit(node.rightNode)):
                    return self.visit(node.leftNode)
                 return ""     
            case TokenTypes.DOT:
                return [i for i in range(self.visit(node.leftNode), self.visit(node.rightNode)+1)]

    def visit_numberNode(self, node):
        return node.value
    
    def visit_forNode(self, node):
        pass

    def visit_scopeNode(self, node):
        pass

    def visit_flexibleNode(self, node):
        pass

class Sequentor:
    def __init__(self, nodes:list[BaseNode]) -> None:
        self.nodes:list[BaseNode] = nodes
        self.others = []
        self.choices = []
        self.pointer_pos = 0

    def getSequences(self):
        self.setUpNodes()
        sequences = self.createSequences() 
        return sequences 

    def setUpNodes(self):
        row = 0
        for n in self.nodes:
            node = n.visit(n)
            if node.node.token.type == TokenTypes.CHOICE:
                self.choices.append([row, copy.deepcopy(node.node)]) # Deep copy since duplicates can change value during operation of functions.
            elif node.node.token.type == TokenTypes.FAIL:
                return FailNode(TokenTypes.FAIL, TokenTypes.FAIL.value)
            else: self.choices.append([row, node.node])
            row += 1

   

# Sets up the list with values that are fix (Not choices)
    def setUp(self):
        cv_fix = list(range(0, len(self.choices)))
        for o in self.others:
            cv_fix[o[0]] = o[1]
        return cv_fix

    def createSequences(self):
            current_pointer_pos = len(self.choices) - 2
            last_index = len(self.choices) - 1
            choice_values = []
            current_index = 0
            hasCompleted:bool = False
            point_move_next_choice = False
            hasManyChoices = True

            if(current_pointer_pos < 0):
                current_pointer_pos = 0
                hasManyChoices = False

            while hasCompleted == False:
                choiceVals = self.setUp()
                for c in self.choices:
                
                    current_choice = c[1]

                    if current_index == current_pointer_pos: 
                        if point_move_next_choice :
                            has_next_choice = current_choice.nextChoice()
                            if has_next_choice == False:
                                if current_pointer_pos == 0:
                                    hasCompleted = True
                                    break
                                else: current_pointer_pos -= 1
                            point_move_next_choice = False 
                        val = current_choice.getNextVal(hasManyChoices)
                        
                        # This if statement is only for single choices as such: x:= (2,3); (1,x)
                        if hasManyChoices == False and current_choice.hasNextVal() == False:
                            point_move_next_choice = True

                        choiceVals[c[0]] = val 

                    elif  current_index == last_index:
                        val = None
                        val = current_choice.getNextVal(False)
                        has_next_choice = True
                        if val != None:
                            choiceVals[c[0]] = val
                            
                        if current_choice.hasNextVal() == False:
                            has_next_choice = current_choice.nextChoice()
                        if(has_next_choice == False):
                                point_move_next_choice = True  
                                current_choice.setChoiceBack() 
                    else:
                        val = current_choice.getNextVal(True)    
                        choiceVals[c[0]] = val 

                    current_index += 1
                if(hasCompleted == False):
                    choice_values.append(SequenceNode(Token(TokenTypes.TUPLE_TYPE,TokenTypes.TUPLE_TYPE.value),choiceVals))
                current_index = 0

            return choice_values

class BlockNode(BaseNode):
    def __init__(self, nodes:list[BaseNode]) -> None:
        self.nodes:list[BaseNode] = nodes

class ProgramNode(BaseNode):
    def __init__(self, node:BlockNode) -> None:
        self.node = node

class ScopeNode(BaseNode):
    def __init__(self,token:Token, nodes:list[BaseNode]) -> None:
        super().__init__(token)
        self.nodes = nodes

class BindingNode(BaseNode):
    def __init__(self,token:Token, leftNode:BaseNode, rightNode:BaseNode) -> None:
        super().__init__(token)
        self.leftNode = leftNode
        self.rightNode = rightNode

class NumberNode(BaseNode):
    def __init__(self, token:Token) -> None:
        super().__init__(token)
        self.value = token.value

class OperatorNode(BaseNode):
    def __init__(self, token:Token, leftNode: BaseNode, rightNode: BaseNode) -> None:
        super().__init__(token)
        self.leftNode = leftNode
        self.rightNode = rightNode

class StatementNode(BaseNode):
     def __init__(self, token:Token, leftNode: BaseNode, rightNode: BaseNode) -> None:
        super().__init__(token)
        self.leftNode = leftNode
        self.rightNode = rightNode

class UnaryNode(BaseNode):
     def __init__(self, token:Token, node) -> None:
        super().__init__(token)
        self.node = node

class IdentifierNode(BaseNode):
    def __init__(self, token:Token) -> None: #Change into Variable/IdentifierNode
        super().__init__(token)
        
class ScopeNode(BaseNode):
    def __init__(self, token:Token, nodes:list[BaseNode], type) -> None: #Change into Variable/IdentifierNode
        super().__init__(token)
        self.nodes = nodes
        self.type = type

class TypeNode(BaseNode):
    def __init__(self, token:Token) -> None: 
        super().__init__(token)
        self.type = type

class TypeNodeSequence(TypeNode):
    def __init__(self, token:Token, types:list[TypeNode]) -> None: 
        super().__init__(token)
        self.types = types


class ArgumentsNode: 
    def __init__(self, nodes:list[BaseNode]) -> None: 
        self.nodes = nodes

class FuncCallNode:
    def __init__(self,identifier:IdentifierNode, args:ArgumentsNode) -> None:
        self.identifier = identifier
        self.args = args

class ParamsNode:
    def __init__(self, nodes:list[ScopeNode]) -> None:
        self.nodes = nodes

class FuncDeclNode:
    def __init__(self,identifier:IdentifierNode, nodes:list[ParamsNode],usesLambda:bool, type:TypeNode, block:BlockNode) -> None:
        self.identifier = identifier
        self.nodes = nodes
        self.usesLambda = usesLambda
        self.type = type
        self.block = block

class ForNode(BaseNode):
     def __init__(self, token:Token, node: BaseNode, condition: BaseNode, expr: BaseNode, do: BaseNode) -> None:
        super().__init__(token)
        self.node = node
        self.condition = condition
        self.expr = expr
        self.do = do

class IfNode(BaseNode):
     def __init__(self, token:Token, if_node: BaseNode, then_node: BaseNode, else_node: BaseNode) -> None:
        super().__init__(token)
        self.if_node = if_node
        self.then_node = then_node
        self.else_node = else_node

class RigidEqNode(BaseNode):
     def __init__(self, token:Token, left_node:BaseNode, right_node:BaseNode) -> None:
        super().__init__(token)
        self.left_node = left_node
        self.right_node = right_node

class FlexibleEqNode(BaseNode):
     def __init__(self, token:Token, left_node:BaseNode, right_node:BaseNode) -> None:
        super().__init__(token)
        self.left_node = left_node
        self.right_node = right_node

class SequenceNode(BaseNode):
     def __init__(self, token:Token, nodes:list[BaseNode]) -> None:
        super().__init__(token)
        self.nodes = nodes
        self.seperator = ","
        
     def __repr__(self) -> str:    
        return self.seperator.join([repr(n) for n in self.nodes])

class IndexingNode(BaseNode):
      def __init__(self, token:Token,identifier:IdentifierNode, index:BaseNode) -> None:
        super().__init__(token)
        self.identifier = identifier
        self.index = index

class ChoiceSequenceNode(TypeNode):
    def __init__(self, token:Token, nodes:list[BaseNode]) -> None:
        super().__init__(token)
        self.nodes = nodes
        self.seperator = "|"
        
        self.currentChoice:int = 0 
        self.currentVal:int = - 1 
        self.nodes = nodes
        self.getVals = self.getValsOfChoice()
        
    def __repr__(self) -> str:    
        return self.seperator.join([repr(n) for n in self.nodes])
    
    def getValsOfChoice(self):
        val = []
        current_node = self.nodes[self.currentChoice]
        if self.currentChoice > len(self.nodes) - 1:
            return None
        if current_node.token.type == TokenTypes.CHOICE:
            for v in current_node.yieldVal():
                val.append(v)
            return val
        else: return [current_node]

    def nextChoice(self) -> bool:
        if self.currentChoice + 1 > len(self.nodes) - 1:
            self.setChoiceBack()
            self.setValCountBack()
            return False
        else: 
            self.currentChoice += 1
            self.setValCountBack()
            self.getVals = self.getValsOfChoice()         
            return True     

    def setChoiceBack(self):
        self.currentChoice = 0
        self.setValCountBack()
        self.getVals = self.getValsOfChoice()

    def setValCountBack(self):
        self.currentVal = -1
    
    def getNextVal(self, repeat:bool):
        if self.currentVal + 1 > len(self.getVals) - 1 and repeat: # If first val none, maybe error
            self.setValCountBack()
            self.currentVal += 1
            return self.getVals[self.currentVal]
        elif self.currentVal + 1 > len(self.getVals) - 1: # Need to delete or change this line of code (Not needed)
            return None
        else:
            self.currentVal += 1
            return self.getVals[self.currentVal]
        
    def hasNextVal(self):
        nextValExists = self.currentVal < len(self.getVals) - 1
        return nextValExists

    def yieldVal(self):
        for c in self.nodes:   
            if c.token.type == TokenTypes.CHOICE:
                yield c.yieldVal()
            else: yield c


class FailNode(BaseNode): # Technically not need, since Fail node is 1 to 1 a BaseNode
      def __init__(self, token:Token) -> None:
        super().__init__(token)

# Symboltabelle
contains the information of current variables
used imports:
- `namedtuple`

In [33]:
class Symboltable:
    def __init__(self) -> None:
        self.symboltable = []
    
    def __info__(self) -> None:
        for t in self.symboltable:
            print("Symboltable: Name= {}, Value= {}, ParentNode= {}".format(t[0], t[1], t[2]))
    
    def check_if_exists(self, name: string, parentNode: BaseNode) -> bool:
        for symbol in self.symboltable:
            if symbol[0] == name:
                if symbol[2] == parentNode:
                    return True 
        return False
    
    def add(self, name: string, value: BaseNode, parentNode: BaseNode) -> None:
        # checks if the name already exists in the current scope. Otherwise add to table.
        if self.check_if_exists(name, parentNode) == False:
            self.symboltable.append([name, value, parentNode])
    
    def remove(self, name:string, value: BaseNode, parentNode: BaseNode) -> None:
        # checks if the table is empty.
        if len(self.symboltable) < 1:
            return
        
        # iterates through and removes the corresponding 
        for symbol in self.symboltable:
            if symbol[0] == name:
                if symbol[2] == parentNode:
                    self.symboltable.remove([name, value, parentNode]) 
    
    def get_value(self, name: string, parentNode: BaseNode) -> tuple[bool, BaseNode]:
        for symbol in self.symboltable:
            if symbol[0] == name:
                if symbol[2] == parentNode:
                    return True, symbol[1]
        return False, parentNode



## Symboltable tests

In [34]:
table = Symboltable()
parentNode = NumberNode(Token(TokenTypes.INTEGER, 6))
table.remove("x", NumberNode(Token(TokenTypes.INTEGER, 7)), parentNode)
table.add("y", NumberNode(Token(TokenTypes.INTEGER, 7)), parentNode)
valid, value = table.get_value("y", parentNode)
if valid:
    print( "y: " + str(value.visit(value)))

y: 7


# Scopetable

In [35]:
class Scope:
    def __init__(self, symbol: string, value: BaseNode, symbolType: TokenTypes, insideTable) -> None:
        self.symbol: string = symbol
        self.value: BaseNode | None = value
        self.symbolType: TokenTypes | None = symbolType
        self.insideTable = insideTable

class ScopeTable:
    def __init__(self) -> None:
        self.scopetable: list[Scope] = []
    
    def __info__(self) -> None:
        for scope in self.scopetable:
            print("Symboltable: Name= {}, Value= {}, type= {} and inside table={}".format(scope.symbol, scope.value, scope.symbolType, scope.insideTable))
    
    def check_if_exists(self, symbol: string, table) -> bool:
        for scope in self.scopetable:
            if scope.symbol == symbol and table == scope.insideTable:
                return True
        return False
    
    def addScope(self, symbol: string, value: BaseNode, symbolType: TokenTypes) -> None:
        # checks if the name already exists in the current scope. Otherwise add to table.
        if self.check_if_exists(symbol, self) == False:
            self.scopetable.append(Scope(symbol, value, symbolType, self))
        
        # checks if the scope is already defined with type or value.
        for scope in self.scopetable:
            if scope.symbol == symbol and scope.symbolType != None and scope.value == None and value != None:
                scope.value = value
            elif scope.symbol == symbol and scope.symbolType == None and scope.value != None and symbolType != None:
                scope.symbolType = symbolType
    
    def addScopeTable(self, scopetable) -> None:

            # checks if the name already exists in the current scope. Otherwise add to table.
            for scope in scopetable.scopetable:
                if self.check_if_exists(scope.symbol, scopetable) == False:
                    self.scopetable.append(Scope(scope.symbol, scope.value, scope.symbolType, scopetable))

    
    def remove(self, symbol:string, value: BaseNode, symbolType: type) -> None:
        # checks if the table is empty.
        if len(self.scopetable) < 1:
            return
        
        # iterates through and removes the corresponding 
        for scope in self.scopetable:
            if scope.symbol == symbol:
                if scope.insideTable == self:
                    self.scopetable.remove(Scope(scope.symbol, scope.value, scope.symbolType, self)) 
    
    def get_value(self, symbol: string, scopetable) -> tuple[bool, BaseNode]:
        for scope in self.scopetable:
            if scope.symbol == symbol:
                if scope.insideTable == scopetable:
                    return True, scope.value
        return False, None

In [36]:
table = ScopeTable()
table.addScope("y", NumberNode(Token(TokenTypes.INTEGER, 7)), TokenTypes.INT_TYPE)

table2 = ScopeTable()
table2.addScope("x", NumberNode(Token(TokenTypes.INTEGER, 4)), TokenTypes.INT_TYPE)

table.addScopeTable(table2)

table.__info__()

Symboltable: Name= y, Value= <__main__.NumberNode object at 0x00000296D05B9A90>, type= TokenTypes.INT_TYPE and inside table=<__main__.ScopeTable object at 0x00000296D0299410>
Symboltable: Name= x, Value= <__main__.NumberNode object at 0x00000296D05B9710>, type= TokenTypes.INT_TYPE and inside table=<__main__.ScopeTable object at 0x00000296D0121BD0>


# Parser

In [37]:
class Parser:
    def __init__(self, lexer: lexicon):
       self.logger: Logger = Console_Logger()
       self.end = False
       self.lexer = lexer
       self.current_token = lexer.get_token(self.lexer.current_char)
       self.scopetable = ScopeTable()
       

    def parse(self) -> BaseNode:     
        node = self.program()
        if node.hasSyntaxError or self.current_token.type != TokenTypes.EOF:
            self.logger.__log_error__("it appears there was a problem", ErrorType.SyntaxError)
        return node.node
    

       
    #####################################
    # statements
    #####################################


    """
    Checks if the program has either statements or function calls.
    Rule => statement_list | func_decl (SEMI statement_list | func_decl)*?
    """
    def program(self) -> ParsedNode:
        
        index = self.lexer.index
        token = self.current_token
        
        node = self.func_decl()

        # checks if start of the program is not a block but a function.
        if(node.hasSyntaxError == True):
            self.set_to_token(index, token)
            node = self.block()
        
        # checks if the program is not a function either.
        if(node.hasSyntaxError == True):
            return ParsedNode(None, True)
        
        nodes = node
        
        return ParsedNode(ProgramNode(nodes.node), False)
    
    

    """
    Checks if the program has a list of statements.
    Rule => statement_list
    """
    def block(self) -> ParsedNode:
        nodes = self.statement_list()
        if(nodes[0].hasSyntaxError):
            return ParsedNode(None, True)
        return ParsedNode(BlockNode(nodes), False)



    """
    Checks if there are one or more statements.
    Rule => statement (SEMI statement)*?   
    """
    def statement_list(self) -> list[ParsedNode]:
        nodes = []
        nodes.append(self.statement())

        while True:
            if(self.current_token.type == TokenTypes.SEMICOLON):
                self.forward()
                nodes.append(self.statement())
            else:
                for node in nodes:
                    if node.hasSyntaxError:
                        return [ParsedNode(None, True)]
                break


        return nodes



    """
    Checks if the current node is either an expression statemnts, if statement, for statement, a function call or an assignment statemnt.
    Rule => expr | if | for | func_call | assign_statement
    """
    def statement(self) -> ParsedNode:
        token = self.current_token
        index = self.lexer.index

        node: ParsedNode = self.nested_scope()

        # checks if current node is not an expression.
        if(node.hasSyntaxError == True):
             self.set_to_token(index,token)
             node = self.flexible_eq()
             
        if(node.hasSyntaxError == True):
            self.set_to_token(index,token)
            node = self.expr()
        
        return node
    
    """
    Flexible Eq Statement (Used only to give something a value)
    Rule -> (Identifier EQUAL expr)
    """
    def flexible_eq(self)-> ParsedNode:
        left_node = self.identifier()
        if(left_node.hasSyntaxError == False):
            if(self.current_token.type == TokenTypes.EQUAL):
                token = self.current_token
                self.forward()
                right_node = self.expr()
                if(right_node.hasSyntaxError == False):
                    return ParsedNode(FlexibleEqNode(token,left_node.node,right_node.node), False)
        return ParsedNode(None,True)


    """
    Rigid Eq Statement (Used only to check if two expr are equals)
    Rule -> (expr (EQUAL expr)*?)
    """
    def rigid_eq(self) -> ParsedNode:
        left_node = self.expr()
        if(left_node.hasSyntaxError == False):
            if(self.current_token.type == TokenTypes.EQUAL):
                node = ParsedNode(None,True)
                while self.current_token.type == TokenTypes.EQUAL:
                    token = self.current_token
                    self.forward()
                    right_node = self.expr()
                    if(right_node.hasSyntaxError == False):
                        if(node.node == None):
                                node = RigidEqNode(token,left_node.node,right_node.node)
                        if(self.current_token.type == TokenTypes.EQUAL):
                            node = RigidEqNode(token, node, right_node)
                    else: return ParsedNode(None,True)
                return ParsedNode(node,False)
            return left_node
        return ParsedNode(None,True)
   
  
    """
    Checks for a function call.
    Rule => IDENTIFIER LB (func_call_args)? RB 
    """
    def func_call(self) -> ParsedNode:
        # RULE --> IDENTIFIER LB (func_call_param)? RB  NOT IMPLEMENTED
        node = self.identifier()
        if(node.hasSyntaxError):
            return ParsedNode(None, True)  
        if(self.current_token.type == TokenTypes.LBRACKET):
            self.forward()
            
            # checks if it is an empty function call.
            if self.current_token.type == TokenTypes.RBRACKET:
                self.forward()
                return ParsedNode(FuncCallNode(node, None), False)

            params = self.func_call_args()
            for param in params:
                if param.hasSyntaxError:
                    return ParsedNode(None, True)
                
            if(self.current_token.type == TokenTypes.RBRACKET):
                self.forward()
                return ParsedNode(FuncCallNode(node, params), False)
                
        return node
        

    """
    Checks for the arguments of the function call.
    Rule => expr (COMMA expr)*?  
    """    
    def func_call_args(self) -> list[ParsedNode]:
        nodes = []
        nodes.append(self.expr())

        while True:
            if(self.current_token.type == TokenTypes.COMMA):
                self.forward()
                nodes.append(self.expr())
            else:
                for node in nodes:
                    if node.hasSyntaxError:
                        return [ParsedNode(None, True)]
                break
        
        return nodes


    """
    Checks for the declaration of a function.
    Rule => IDENTIFIER LB func_dec_param RB (COLON type)? BINDING block
           |IDENTIFIER BINDING LB nested_scope LAMBDA expr RB    
    """
    def func_decl(self) -> ParsedNode:
        identifier = self.identifier()
        if identifier.hasSyntaxError == False:

            if self.current_token.type == TokenTypes.LBRACKET:
                self.forward()

                params = self.func_decl_param()

                if self.current_token.type == TokenTypes.RBRACKET:
                    self.forward()
                    type = None
                    if(self.current_token.type == TokenTypes.COLON):
                        self.forward()
                        type = self.type()
                        if(type.hasSyntaxError):
                            return ParsedNode(None,True)
                        
                    if self.current_token.type == TokenTypes.BINDING:
                        self.forward()
                        block = self.block()
                        if(block.hasSyntaxError == False):
                            return ParsedNode(FuncDeclNode(identifier,params,False,type,block),False)
                return ParsedNode(None,True)
            
            if self.current_token.type == TokenTypes.BINDING:
                self.forward()
                if self.current_token.type == TokenTypes.LBRACKET:
                    self.forward()
                    type = None
                    params = self.func_decl_param()
                    if(params.hasSyntaxError):
                        return ParsedNode(None,True)
                    
                    if self.current_token.type == TokenTypes.LAMBDA:
                        self.forward()                   
                        block = self.block()
                        if(block.hasSyntaxError == False and self.current_token.type == TokenTypes.RBRACKET):
                            self.forward()
                            return ParsedNode(FuncDeclNode(identifier,params,True,type,block),False)
                    return ParsedNode(None,True)

        return ParsedNode(None,True)

    """
    Checks for the arguments of the function declaration.
    Rule => nested_scope
    """
    def func_decl_param(self) -> ParsedNode:
        nodes:list[ScopeNode] = []
        if(self.current_token.type == TokenTypes.RBRACKET):
            return ParsedNode(ParamsNode(nodes),False)
        
        node = self.scope()
        if(node.hasSyntaxError == False):
            nodes.append(node.node)
            while(self.current_token.type == TokenTypes.COMMA):
                self.forward()
                node = self.scope()
                if(node.hasSyntaxError):
                    return ParsedNode(None,True)
                nodes.append(node.node)
            return ParsedNode(ParamsNode(nodes),False)
        return ParsedNode(None,True)

    """
    Checks if the statement is an if statement.
    Rule => IF LB expr RB THEN CBL block CBR ELSE CBL block CBR
          | IF LB expr RB THEN expr ELSE expr  
    """
    def if_statement(self) -> ParsedNode:
        token = self.current_token

        if token.type != TokenTypes.IF:
            return ParsedNode(None, True)
        
        self.forward()
        if self.current_token.type != TokenTypes.LBRACKET:
            return ParsedNode(None, True)
        
        self.forward()
        if_node = self.rigid_eq()
        if if_node.hasSyntaxError == True or self.current_token.type != TokenTypes.RBRACKET:
            return ParsedNode(None, True)
        
        self.forward()
        if self.current_token.type != TokenTypes.THEN:
            return ParsedNode(None, True)
        
        self.forward()

        hasCB:bool = self.current_token.type == TokenTypes.CBL
        then_node = ParsedNode(None,True)
        else_node = ParsedNode(None,True)

        if(hasCB):
            self.forward()
            then_node = self.block()
            if(then_node.hasSyntaxError):
                return ParsedNode(None,True)
            if(self.current_token.type == TokenTypes.CBR):
                self.forward()
            else: return ParsedNode(None,True)
        else:
            then_node = self.expr()
            if(then_node.hasSyntaxError):
                return ParsedNode(None,True)
            
        if self.current_token.type != TokenTypes.ELSE:
            return ParsedNode(None, True)     
        self.forward()

        if(hasCB and self.current_token.type == TokenTypes.CBL):
            self.forward()
            else_node = self.block()
            if(then_node.hasSyntaxError):
                return ParsedNode(None,True)
            if(self.current_token.type == TokenTypes.CBR):
                self.forward()
            else: return ParsedNode(None,True)
        else:
            else_node = self.expr()
            if(then_node.hasSyntaxError):
                return ParsedNode(None,True)
            
        return ParsedNode(IfNode(token, if_node.node, then_node.node, else_node.node),False)
            

    """
    Checks if the statement is a loop expression.
    Rule => FOR CBL (scope|expr) (;expr)*? CBR
          | FOR LB (scope|expr) (,expr)*? RB DO expr
    """
    def for_loop(self) -> ParsedNode:
        token = self.current_token

        if token.type != TokenTypes.FOR:
            return ParsedNode(None, True)
        
        self.forward()
        # checks if the for loop is defined with a curly bracket.
        if self.current_token.type == TokenTypes.CBL:
            self.forward()
            return self.for_loop_curly()
        elif self.current_token.type == TokenTypes.LBRACKET:
            self.forward()
            return self.for_loop_bracket()

        return ParsedNode(None, True)
            
    
    def for_loop_curly(self) -> ParsedNode:
        condition: ParsedNode
        expression: ParsedNode
        token = self.current_token
        index = self.lexer.index

        node = self.scope()

        # checks if the loop content is not a scope but an expression.
        if node.hasSyntaxError == True:
            self.set_to_token(index, token)
            node = self.expr()
        
        # checks if the loop input is invalid.
        if node.hasSyntaxError:
            return ParsedNode(None, True)
        
        if self.current_token.type == TokenTypes.CBR:
            self.forward()
            return ParsedNode(ForNode(TokenTypes.FOR, node=node.node, condition=None, expr=None, do=None), False)
        
        self.forward()
        token = self.current_token
        index = self.lexer.index
        condition = self.flexible_eq()

        if(condition.hasSyntaxError == True):
            self.set_to_token(index,token)
            condition = self.expr()

        if condition.hasSyntaxError:
            return ParsedNode(None, True)

        if self.current_token.type == TokenTypes.SEMICOLON:
            self.forward()
            token = self.current_token
            index = self.lexer.index
            expression = self.flexible_eq()

            if(expression.hasSyntaxError == True):
                self.set_to_token(index,token)
                expression = self.expr()
        
            if self.current_token.type != TokenTypes.CBR or expression.hasSyntaxError:
                return ParsedNode(None, True)

            self.forward()
            return ParsedNode(ForNode(TokenTypes.FOR, node=node.node, condition=condition.node, expr=expression.node, do=None),False)
        
        if self.current_token.type == TokenTypes.CBR:
            self.forward()
            return ParsedNode(ForNode(TokenTypes.FOR, node=node.node, condition=None, expr=condition.node, do=None), False)
        
        return ParsedNode(None, True)
    
    def for_loop_bracket(self) -> ParsedNode:
        condition: ParsedNode
        expression: ParsedNode
        token = self.current_token
        index = self.lexer.index

        node = self.scope()

        # checks if the loop content is not a scope but an expression.
        if node.hasSyntaxError == True:
            self.set_to_token(index,token)
            node = self.expr()
        
        # checks if the loop input is invalid.
        if node.hasSyntaxError:
            return ParsedNode(None, True)
        
        if self.current_token == TokenTypes.RBRACKET:
            self.forward()
            return ParsedNode(ForNode(TokenTypes.FOR, node=node.node, condition=None, expr=None, do=None), False)
        
        self.forward()
        token = self.current_token
        index = self.lexer.index
        condition = self.flexible_eq()

        if(condition.hasSyntaxError == True):
            self.set_to_token(index,token)
            condition = self.expr()

        if condition.hasSyntaxError:
            return ParsedNode(None, True)

        if self.current_token.type == TokenTypes.SEMICOLON:
            self.forward()
            token = self.current_token
            index = self.lexer.index
            expression = self.flexible_eq()

            if(expression.hasSyntaxError == True):
                self.set_to_token(index,token)
                expression = self.expr()
        
            if self.current_token.type != TokenTypes.RBRACKET or expression.hasSyntaxError:
                return ParsedNode(None, True)
        
            self.forward()
            if self.current_token.type != TokenTypes.DO:
                return ParsedNode(None, True)
        
            self.forward()
            do = self.expr()
        
            if do.hasSyntaxError:
                return ParsedNode(None, True)
            
            return ParsedNode(ForNode(token, node=node.node, condition=condition.node, expr=expression.node, do=do.node), False)
        
        if self.current_token.type != TokenTypes.RBRACKET:
                return ParsedNode(None, True)
        
        self.forward()
        if self.current_token.type != TokenTypes.DO:
            return ParsedNode(None, True)
        
        self.forward()
        do = self.expr()
        
        if do.hasSyntaxError:
            return ParsedNode(None, True)
            
        return ParsedNode(ForNode(token, node=node.node, condition=None, expr=condition.node, do=do.node), False)


    """
    Checks if the statement is a nested scope.
    Rule =>  (Identifier COMMA Identifier)* COLON TYPE
    """
    def nested_scope(self) -> ParsedNode:
        nodes: list[ParsedNode] = []
        nodes.append(self.identifier())

        if nodes[0].hasSyntaxError:
            return ParsedNode(None, True)
            
        hasComma:bool = False
        while True:        
            if self.current_token.type == TokenTypes.COMMA:
                hasComma = True
                self.forward()
                nodes.append(self.identifier())
            else:
                for node in nodes:
                    if node.hasSyntaxError:
                        return ParsedNode(None, True)
                break
        
        if hasComma == False or self.current_token.type != TokenTypes.COLON:
            return ParsedNode(None, True)
        
        self.forward()
        type = self.type()

        if type.hasSyntaxError == True:
            return ParsedNode(None, True)

        return ParsedNode(ScopeNode(TokenTypes.COLON, nodes, type), False)
        

    #####################################
    # expressions
    #####################################

    def expr(self) -> ParsedNode:         
        leftNode = self.operation()

        if leftNode.hasSyntaxError:
            return ParsedNode(None, True)

        if self.current_token.type != TokenTypes.DOT:
            return leftNode
        
        self.forward()
        if self.current_token.type != TokenTypes.DOT:
            return ParsedNode(None, True)
        
        token = self.current_token
        self.forward()
        rightNode = self.operation()

        if rightNode.hasSyntaxError:
            return ParsedNode(None, True)

        return ParsedNode(OperatorNode(token, leftNode.node, rightNode.node), False)
    
    def choice(self):
        node = self.operation()
        token = self.current_token

        nodes:list[BaseNode] = []
     
        if(node.hasSyntaxError==False and self.current_token.type == TokenTypes.CHOICE):
                nodes.append(node.node)
                while(self.current_token.type == TokenTypes.CHOICE):
                    token = self.current_token
                    self.forward()
                    node = self.operation()
                    if(node.hasSyntaxError==False):
                        nodes.append(node.node)
                    else: return ParsedNode(None,True)
                return ParsedNode(ChoiceSequenceNode(token,nodes), False)
        return node
    
    """
    This method checks if a token any of the following operations: =, <, >, <=, >=, |, +, -
    Since all of this operations have the same priority and same values output, it is not needed to write them in different methods
    """
    def operation(self):
        # RULE --> op: term ((GT|LT|GE|LE|EQUAL|CHOICE|PLUS|MINUS) term)*

        left_node = self.term()

        # Checks if left node has been received and if the following token is one of the following tokens: : =, <, >, <=, >=, |, +, -

        if(left_node.hasSyntaxError == False and (self.check_type(self.current_token.type,
                [TokenTypes.GREATER,TokenTypes.GREATEREQ,TokenTypes.LOWER,TokenTypes.LOWEREQ, TokenTypes.PLUS,
                TokenTypes.MINUS]))):

                node = ParsedNode(None,True)
                
                # The while method "concatenates" the operations

                while(self.check_type(self.current_token.type,
                [TokenTypes.GREATER,TokenTypes.GREATEREQ,TokenTypes.LOWER,TokenTypes.LOWEREQ, TokenTypes.PLUS,
                TokenTypes.MINUS])):
                
                    token = self.current_token
                    self.forward()
                    right_node = self.term()
                    if(right_node.hasSyntaxError):
                        return right_node
                    
                    # Binds found operation to its left node
                    if(node.node == None):
                       node = ParsedNode(OperatorNode(token,left_node.node,right_node.node),False)
                    else: node = ParsedNode(OperatorNode(token,node.node,right_node.node),False)
                return node
        return left_node

    """
    Checks the same way in operation method but here it checks for *, /
    """
    def term(self) -> ParsedNode:
        # RULE --> factor ((MUL|DIV) factor)*
        
        left_node = self.factor() 

        if(left_node.hasSyntaxError == False and (self.check_type(self.current_token.type,[TokenTypes.MULTIPLY, TokenTypes.DIVIDE]))):
            node = ParsedNode(None,True)

             # The while method "concatenates" the operations
            while(self.check_type(self.current_token.type,[TokenTypes.MULTIPLY, TokenTypes.DIVIDE])):
               
                token = self.current_token
                self.forward()
                right_node = self.factor()
                if(right_node.hasSyntaxError):
                    return right_node
                
                # Binds found operation to its left node
                if(node.node == None):
                  node = ParsedNode(OperatorNode(token,left_node.node,right_node.node),False)
                else: node = ParsedNode(OperatorNode(token,node.node,right_node.node),False)
            return node
        return left_node
    
    """
    Checks for unary operations, Integers, brackets (highest priority)
    RULE -->  INTEGER  
        : brackets
        : (MINUS|PLUS) arith
        : func_call x() x
        : indexing     NOT IMPLEMENTING
        : --> means the same as (brackets|unary|func_call) just like in operation()
        only that for each if a different Node may be created not such as only OperationNode like in operation()
    """
    def factor(self) -> ParsedNode:
        token = self.current_token
        index = self.lexer.index

        #Integer check
        if(token.type == TokenTypes.INTEGER):
            self.forward()
            return ParsedNode(NumberNode(token),False)
        
        if(token.type == TokenTypes.FAIL):
            self.forward()
            return ParsedNode(FailNode(token),False)
        
        #Unary operation check
        if(self.check_type(self.current_token.type,[TokenTypes.PLUS, TokenTypes.MINUS])):
            self.forward()
            node = self.operation()
            if(node.hasSyntaxError):        # (--) --> Error needs (-- expr) or (--3)
                return ParsedNode(None, True)
            return ParsedNode(UnaryNode(token,node.node),False)
        
        #brackets check
        node = self.brackets()

        #if node has failed check for loop
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.indexing()

        #if node has failed check for loop
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.scope()

         #if node has failed check for loop
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.binding()

        #if node has failed check indexing
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.func_call()
        
        #if node has failed check for loop
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.for_loop()

        #if node has failed check if statement
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.if_statement()
        
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.sequence()
            
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.identifier()
        return node
    
    """
    Checks for brackets (highest priority)
    RULE --> brackets: LB expr RB
    """
    def brackets(self) -> ParsedNode: 
        if(self.current_token.type == TokenTypes.LBRACKET):
            self.forward()
            node = self.block()
        
            if(self.current_token.type == TokenTypes.RBRACKET):
                self.forward()
                return node
        return ParsedNode(None,True)
        


    """
    y := 8 y:=(x:int)  y:= method(...)...
    RULE --> scope BINDING expr
    """
    def binding(self) -> ParsedNode:
        left_node = self.identifier()

        if(left_node.hasSyntaxError == False):
            if(self.check_type(self.current_token.type,[TokenTypes.BINDING])):
                token = self.current_token
                self.forward()
                right_node = self.expr()
                if(right_node.hasSyntaxError == False):
                    return ParsedNode(BindingNode(token,left_node.node,right_node.node), False)
                else: return ParsedNode(None,True)
        return ParsedNode(None,True)


    """
    x:int
    RULE --> identifier COLON type 
    """
    def scope(self) -> ParsedNode:
        left_node = self.identifier()
        if(left_node.hasSyntaxError == False):
            if(self.check_type(self.current_token.type,[TokenTypes.COLON])):
                token = self.current_token
                self.forward()
                type = self.type()
                if(type.hasSyntaxError == False):
                    return ParsedNode(ScopeNode(token,[left_node.node],type.node),False) # Return Scope Node
                else: return ParsedNode(None,True)
            if self.check_type(self.current_token.type, [TokenTypes.BINDING]):
                token = self.current_token
                
        return ParsedNode(None,True)
    

    """
    variable/method name
    RULE --> identifier            NEED UPDATE
    """
    def identifier(self) -> ParsedNode:
        token = self.current_token
        if(token.type == TokenTypes.IDENTIFIER):
            self.forward()
            return ParsedNode(IdentifierNode(token), False)
        return ParsedNode(None, True) 
        
    """
    int or tuple(int,int) or array{int}
    RULE -->  INT                        
            : TUPLE LB type (,type)* RB 
    """
    def type(self) -> ParsedNode:
        # RULE -->  INT                        
        #        : TUPLE LB type (,type)* RB    

        token = self.current_token
        if(token.type == TokenTypes.INT_TYPE):
            self.forward()
            return ParsedNode(TypeNode(token),False)
        
        if(token.type == TokenTypes.TUPLE_TYPE):
            self.forward()
            if(self.current_token.type == TokenTypes.LBRACKET):
                self.forward()
                types:list[TypeNode] = []

                type = self.type()
                if(type.hasSyntaxError == False):
                    types.append(type.node)
                    if(self.check_type(self.current_token.type, [TokenTypes.COMMA])):
                        while(self.current_token.type == TokenTypes.COMMA):

                            self.forward()
                            t = self.type()

                            if(t.hasSyntaxError):  #If on error
                                return ParsedNode(None,True)
                            types.append(t.node) #else append to list of types
                     
            if(self.current_token.type == TokenTypes.RBRACKET):  
                self.forward()
                return ParsedNode(TypeNodeSequence(TokenTypes.TUPLE_TYPE,types), False)
            
        return ParsedNode(None, True) 
        
    """
    a[i:int]
    # RULE --> identifier SBL expr SBR
    """
    def indexing(self) -> ParsedNode:
        left_node = self.identifier()
        if(left_node.hasSyntaxError == False):
            if(self.current_token.type == TokenTypes.SBL):
                self.forward()
                expr_node = self.expr()
          
                if(expr_node.hasSyntaxError == False and self.current_token.type == TokenTypes.SBR):
                     self.forward()
                     return ParsedNode(IndexingNode(left_node.node.token,left_node.node,expr_node.node),False)
                return ParsedNode(None,True)
        return ParsedNode(None,True)
    
    """
    RUKE --> LB (expr COMMA expr)* RB                  --> tuple (n1,...)
             array CBL expr (COMMA expr)*? CBR         --> long-form syntax and singleton tuple/array array{n1} oder array{n1,...}
    """
    def sequence(self) -> ParsedNode:
        token = self.current_token

        nodes:list[BaseNode] = []

        # Tuple
        if(token.type == TokenTypes.LBRACKET):
            self.forward()
            node = self.expr()
            if(node.hasSyntaxError==False):
                nodes.append(node.node)
                while(self.current_token.type == TokenTypes.COMMA):
                    self.forward()
                    node = self.expr()
                    if(node.hasSyntaxError==False):
                        nodes.append(node.node)
                    else: return ParsedNode(None,True)
                if(len(nodes) > 1 and self.current_token.type == TokenTypes.RBRACKET):
                    self.forward()
                    return ParsedNode(SequenceNode(Token(TokenTypes.TUPLE_TYPE,TokenTypes.TUPLE_TYPE.value),nodes), False)


        # Array
        if(token.type == TokenTypes.ARRAY_TYPE):
                self.forward()
                if(self.current_token.type == TokenTypes.CBL):
                    self.forward()
                    node = self.expr()
                    if(node.hasSyntaxError == False):
                        nodes.append(node.node)
                        while(self.current_token.type == TokenTypes.COMMA):
                              self.forward()
                              node = self.expr()
                              if(node.hasSyntaxError==False):
                                nodes.append(node.node)
                              else: return ParsedNode(None,True)
                        if(self.current_token.type == TokenTypes.CBR):
                            self.forward()
                            return ParsedNode(SequenceNode(Token(TokenTypes.TUPLE_TYPE,TokenTypes.TUPLE_TYPE.value),nodes), False)
                        
        return ParsedNode(None, True)
                

    """
    Moves forward in the tokens list
    """
    def forward(self) -> None:
        print(self.current_token.__info__())
        self.lexer.forward()
        self.current_token = lexer.get_token(self.lexer.current_char)
        if self.current_token.type == TokenTypes.EOF:
            self.end = True
        

        
    """
    Checks if a type exists in the following types list
    """
    def check_type(self,type:TokenTypes,types:list[TokenTypes]) -> bool:
        return type in types
    

    """
    Sets current token back if a certain path lead to failure (Wrong syntax)
    May need it for later
    """
    def set_to_token(self,index, token): 
        self.current_token = token
        self.lexer.index = index

In [38]:
class Parser:
    def __init__(self, lexer: lexicon):
       self.logger: Logger = Console_Logger()
       self.end = False
       self.lexer = lexer
       self.current_token = lexer.get_token(self.lexer.current_char)
       

    def parse(self) -> BaseNode:     
        node = self.program()
        if node.hasSyntaxError or self.current_token.type != TokenTypes.EOF:
            self.logger.__log_error__("it appears there was a problem", ErrorType.SyntaxError)
        return node.node
    

       
    #####################################
    # statements
    #####################################


    """
    Checks if the program has either statements or function calls.
    Rule => statement_list | func_decl (SEMI statement_list | func_decl)*?
    """
    def program(self) -> ParsedNode:
        node = self.block()
        
        # checks if the program is not a function either.
        if(node.hasSyntaxError == True):
            return ParsedNode(None, True)
        
        return ParsedNode(ProgramNode(node.node), False)
    
    

    """
    Checks if the program has a list of statements.
    Rule => statement_list
   
    def blockdelete(self) -> ParsedNode:
        
        nodes = self.statement_list()
        if(nodes[0].hasSyntaxError):
            return ParsedNode(None, True)
        return ParsedNode(BlockNode(nodes), False)
    """


    """
    Checks if there are one or more statements.
    Rule => statement (SEMI statement)*?   
    """
    def block(self) -> ParsedNode:
        node = self.statement()
        nodes = []
        nodes.append(node)

        if(node.hasSyntaxError==False):
            if(self.current_token.type == TokenTypes.SEMICOLON):
                while(self.current_token.type == TokenTypes.SEMICOLON):
                    self.forward()
                    node = self.statement()
                    if node.hasSyntaxError:
                        return ParsedNode(None,True)
                    else: nodes.append(node)
                return ParsedNode(BlockNode(nodes), False)
        return node



    """
    Checks if the current node is either an expression statemnts, if statement, for statement, a function call or an assignment statemnt.
    Rule => expr | if | for | func_call | assign_statement
    """
    def statement(self) -> ParsedNode:
        token = self.current_token
        index = self.lexer.index

        node: ParsedNode = self.nested_scope()

        # checks if current node is not an expression.
        if(node.hasSyntaxError == True):
             self.set_to_token(index,token)
             node = self.flexible_eq()
             
        if(node.hasSyntaxError == True):
            self.set_to_token(index,token)
            node = self.expr()
        
        if(node.hasSyntaxError == True):
            self.set_to_token(index,token)
            node = self.func_decl()

        return node
    
    """
    Flexible Eq Statement (Used only to give something a value)
    Rule -> (Identifier EQUAL expr)
    """
    def flexible_eq(self)-> ParsedNode:
        left_node = self.identifier()
        if(left_node.hasSyntaxError == False):
            if(self.current_token.type == TokenTypes.EQUAL):
                token = self.current_token
                self.forward()
                right_node = self.expr()
                if(right_node.hasSyntaxError == False):
                    return ParsedNode(FlexibleEqNode(token,left_node.node,right_node.node), False)
        return ParsedNode(None,True)


    """
    Rigid Eq Statement (Used only to check if two expr are equals)
    Rule -> (expr (EQUAL expr)*?)
    """
    def rigid_eq(self) -> ParsedNode:
        left_node = self.block()
        if(left_node.hasSyntaxError == False):
            if(self.current_token.type == TokenTypes.EQUAL):
                node = ParsedNode(None,True)
                while self.current_token.type == TokenTypes.EQUAL:
                    token = self.current_token
                    self.forward()
                    right_node = self.block()
                    if(right_node.hasSyntaxError == False):
                        if(node.node == None):
                                node = RigidEqNode(token,left_node.node,right_node.node)
                        if(self.current_token.type == TokenTypes.EQUAL):
                            node = RigidEqNode(token, node, right_node)
                    else: return ParsedNode(None,True)
                return ParsedNode(node,False)
            return left_node
        return ParsedNode(None,True)
   
  
    """
    Checks for a function call.
    Rule => IDENTIFIER LB (func_call_args)? RB 
    """
    def func_call(self) -> ParsedNode:
        # RULE --> IDENTIFIER LB (func_call_param)? RB  NOT IMPLEMENTED
        node = self.identifier()
        if(node.hasSyntaxError):
            return ParsedNode(None, True)  
        if(self.current_token.type == TokenTypes.LBRACKET):
            self.forward()
            
            # checks if it is an empty function call.
            if self.current_token.type == TokenTypes.RBRACKET:
                self.forward()
                return ParsedNode(FuncCallNode(node, None), False)

            params = self.func_call_args()
            for param in params:
                if param.hasSyntaxError:
                    return ParsedNode(None, True)
                
            if(self.current_token.type == TokenTypes.RBRACKET):
                self.forward()
                return ParsedNode(FuncCallNode(node, params), False)
                
        return ParsedNode(None, True)
        

    """
    Checks for the arguments of the function call.
    Rule => expr (COMMA expr)*?  
    """    
    def func_call_args(self) -> list[ParsedNode]:
        nodes = []
        nodes.append(self.expr())

        while True:
            if(self.current_token.type == TokenTypes.COMMA):
                self.forward()
                nodes.append(self.expr())
            else:
                for node in nodes:
                    if node.hasSyntaxError:
                        return [ParsedNode(None, True)]
                break
        
        return nodes


    """
    Checks for the declaration of a function.
    Rule => IDENTIFIER LB func_dec_param RB (COLON type)? BINDING block
           |IDENTIFIER BINDING LB nested_scope LAMBDA expr RB    
    """
    def func_decl(self) -> ParsedNode:
        identifier = self.identifier()
        if identifier.hasSyntaxError == False:

            if self.current_token.type == TokenTypes.LBRACKET:
                self.forward()

                params = self.func_decl_param()

                if self.current_token.type == TokenTypes.RBRACKET:
                    self.forward()
                    type = None
                    if(self.current_token.type == TokenTypes.COLON):
                        self.forward()
                        type = self.type()
                        if(type.hasSyntaxError):
                            return ParsedNode(None,True)
                        
                    if self.current_token.type == TokenTypes.BINDING:
                        self.forward()
                        block = self.block()
                        if(block.hasSyntaxError == False):
                            return ParsedNode(FuncDeclNode(identifier,params,False,type,block),False)
                return ParsedNode(None,True)
            
            if self.current_token.type == TokenTypes.BINDING:
                self.forward()
                if self.current_token.type == TokenTypes.LBRACKET:
                    self.forward()
                    type = None
                    params = self.func_decl_param()
                    if(params.hasSyntaxError):
                        return ParsedNode(None,True)
                    
                    if self.current_token.type == TokenTypes.LAMBDA:
                        self.forward()                   
                        block = self.block()
                        if(block.hasSyntaxError == False and self.current_token.type == TokenTypes.RBRACKET):
                            self.forward()
                            return ParsedNode(FuncDeclNode(identifier,params,True,type,block),False)
                    return ParsedNode(None,True)

        return ParsedNode(None,True)

    """
    Checks for the arguments of the function declaration.
    Rule => nested_scope
    """
    def func_decl_param(self) -> ParsedNode:
        nodes:list[ScopeNode] = []
        if(self.current_token.type == TokenTypes.RBRACKET):
            return ParsedNode(ParamsNode(nodes),False)
        
        node = self.scope()
        if(node.hasSyntaxError == False):
            nodes.append(node.node)
            while(self.current_token.type == TokenTypes.COMMA):
                self.forward()
                node = self.scope()
                if(node.hasSyntaxError):
                    return ParsedNode(None,True)
                nodes.append(node.node)
            return ParsedNode(ParamsNode(nodes),False)
        return ParsedNode(None,True)

    """
    Checks if the statement is an if statement.
    Rule => IF LB expr RB THEN CBL block CBR ELSE CBL block CBR
          | IF LB expr RB THEN expr ELSE expr  
    """
    def if_statement(self) -> ParsedNode:
        token = self.current_token

        if token.type != TokenTypes.IF:
            return ParsedNode(None, True)
        
        self.forward()
        if self.current_token.type != TokenTypes.LBRACKET:
            return ParsedNode(None, True)
        
        self.forward()
        if_node = self.rigid_eq()
        if if_node.hasSyntaxError == True or self.current_token.type != TokenTypes.RBRACKET:
            return ParsedNode(None, True)
        
        self.forward()
        if self.current_token.type != TokenTypes.THEN:
            return ParsedNode(None, True)
        
        self.forward()

        hasCB:bool = self.current_token.type == TokenTypes.CBL
        then_node = ParsedNode(None,True)
        else_node = ParsedNode(None,True)

        if(hasCB):
            self.forward()
            then_node = self.block()
            if(then_node.hasSyntaxError):
                return ParsedNode(None,True)
            if(self.current_token.type == TokenTypes.CBR):
                self.forward()
            else: return ParsedNode(None,True)
        else:
            then_node = self.expr()
            if(then_node.hasSyntaxError):
                return ParsedNode(None,True)
            
        if self.current_token.type != TokenTypes.ELSE:
            return ParsedNode(None, True)     
        self.forward()

        if(hasCB and self.current_token.type == TokenTypes.CBL):
            self.forward()
            else_node = self.block()
            if(then_node.hasSyntaxError):
                return ParsedNode(None,True)
            if(self.current_token.type == TokenTypes.CBR):
                self.forward()
            else: return ParsedNode(None,True)
        else:
            else_node = self.expr()
            if(then_node.hasSyntaxError):
                return ParsedNode(None,True)
            
        return ParsedNode(IfNode(token, if_node.node, then_node.node, else_node.node),False)

    """
    Checks if the statement is a loop expression.
    Rule => FOR CBL (scope|expr) (;expr)*? CBR
          | FOR LB (scope|expr) (,expr)*? RB DO expr
    """
    def for_loop(self) -> ParsedNode:
        token = self.current_token

        if token.type != TokenTypes.FOR:
            return ParsedNode(None, True)
        
        self.forward()
        # checks if the for loop is defined with a curly bracket.
        if self.current_token.type == TokenTypes.CBL:
            self.forward()
            return self.for_loop_curly()
        elif self.current_token.type == TokenTypes.LBRACKET:
            self.forward()
            return self.for_loop_bracket()

        return ParsedNode(None, True)
            
            
    
    def for_loop_curly(self) -> ParsedNode:
        condition: ParsedNode
        expression: ParsedNode
        token = self.current_token
        index = self.lexer.index

        node = self.scope()

        # checks if the loop content is not a scope but an expression.
        if node.hasSyntaxError == True:
            self.set_to_token(index, token)
            node = self.expr()
        
        # checks if the loop input is invalid.
        if node.hasSyntaxError:
            return ParsedNode(None, True)
        
        if self.current_token.type == TokenTypes.CBR:
            self.forward()
            return ParsedNode(ForNode(TokenTypes.FOR, node=node.node, condition=None, expr=None, do=None), False)
        
        self.forward()
        token = self.current_token
        index = self.lexer.index
        condition = self.flexible_eq()

        if(condition.hasSyntaxError == True):
            self.set_to_token(index,token)
            condition = self.expr()

        if condition.hasSyntaxError:
            return ParsedNode(None, True)

        if self.current_token.type == TokenTypes.SEMICOLON:
            self.forward()
            token = self.current_token
            index = self.lexer.index
            expression = self.flexible_eq()

            if(expression.hasSyntaxError == True):
                self.set_to_token(index,token)
                expression = self.expr()
        
            if self.current_token.type != TokenTypes.CBR or expression.hasSyntaxError:
                return ParsedNode(None, True)

            self.forward()
            return ParsedNode(ForNode(TokenTypes.FOR, node=node.node, condition=condition.node, expr=expression.node, do=None),False)
        
        if self.current_token.type == TokenTypes.CBR:
            self.forward()
            return ParsedNode(ForNode(TokenTypes.FOR, node=node.node, condition=None, expr=condition.node, do=None), False)
        
        return ParsedNode(None, True)
    
    def for_loop_bracket(self) -> ParsedNode:
        condition: ParsedNode
        expression: ParsedNode
        token = self.current_token
        index = self.lexer.index

        node = self.scope()

        # checks if the loop content is not a scope but an expression.
        if node.hasSyntaxError == True:
            self.set_to_token(index,token)
            node = self.expr()
        
        # checks if the loop input is invalid.
        if node.hasSyntaxError:
            return ParsedNode(None, True)
        
        if self.current_token == TokenTypes.RBRACKET:
            self.forward()
            return ParsedNode(ForNode(TokenTypes.FOR, node=node.node, condition=None, expr=None, do=None), False)
        
        self.forward()
        token = self.current_token
        index = self.lexer.index
        condition = self.flexible_eq()

        if(condition.hasSyntaxError == True):
            self.set_to_token(index,token)
            condition = self.expr()

        if condition.hasSyntaxError:
            return ParsedNode(None, True)

        if self.current_token.type == TokenTypes.SEMICOLON:
            self.forward()
            token = self.current_token
            index = self.lexer.index
            expression = self.flexible_eq()

            if(expression.hasSyntaxError == True):
                self.set_to_token(index,token)
                expression = self.expr()
        
            if self.current_token.type != TokenTypes.RBRACKET or expression.hasSyntaxError:
                return ParsedNode(None, True)
        
            self.forward()
            if self.current_token.type != TokenTypes.DO:
                return ParsedNode(None, True)
        
            self.forward()
            do = self.expr()
        
            if do.hasSyntaxError:
                return ParsedNode(None, True)
            
            return ParsedNode(ForNode(token, node=node.node, condition=condition.node, expr=expression.node, do=do.node), False)
        
        if self.current_token.type != TokenTypes.RBRACKET:
                return ParsedNode(None, True)
        
        self.forward()
        if self.current_token.type != TokenTypes.DO:
            return ParsedNode(None, True)
        
        self.forward()
        do = self.expr()
        
        if do.hasSyntaxError:
            return ParsedNode(None, True)
            
        return ParsedNode(ForNode(token, node=node.node, condition=None, expr=condition.node, do=do.node), False)

    """
    Checks if the statement is a nested scope.
    Rule =>  (Identifier COMMA Identifier)* COLON TYPE
    """
    def nested_scope(self) -> ParsedNode:
        nodes: list[ParsedNode] = []
        nodes.append(self.identifier())

        if nodes[0].hasSyntaxError:
            return ParsedNode(None, True)
            
        hasComma:bool = False
        while True:        
            if self.current_token.type == TokenTypes.COMMA:
                hasComma = True
                self.forward()
                nodes.append(self.identifier())
            else:
                for node in nodes:
                    if node.hasSyntaxError:
                        return ParsedNode(None, True)
                break
        
        if hasComma == False or self.current_token.type != TokenTypes.COLON:
            return ParsedNode(None, True)
        
        self.forward()
        type = self.type()

        if type.hasSyntaxError == True:
            return ParsedNode(None, True)

        return ParsedNode(ScopeNode(TokenTypes.COLON, nodes, type), False)
        

    #####################################
    # expressions
    #####################################

    def expr(self) -> ParsedNode:         
        leftNode = self.choice()

        if leftNode.hasSyntaxError:
            return ParsedNode(None, True)

        if self.current_token.type != TokenTypes.DOT:
            return leftNode
        
        self.forward()
        if self.current_token.type != TokenTypes.DOT:
            return ParsedNode(None, True)
        
        token = self.current_token
        self.forward()
        rightNode = self.choice()

        if rightNode.hasSyntaxError:
            return ParsedNode(None, True)

        return ParsedNode(OperatorNode(token, leftNode.node, rightNode.node), False)
    
    
    def choice(self):
        node = self.operation()
        token = self.current_token

        nodes:list[BaseNode] = []
     
        if(node.hasSyntaxError==False and self.current_token.type == TokenTypes.CHOICE):
                nodes.append(node.node)
                while(self.current_token.type == TokenTypes.CHOICE):
                    token = self.current_token
                    self.forward()
                    node = self.operation()
                    if(node.hasSyntaxError==False):
                        nodes.append(node.node)
                    else: return ParsedNode(None,True)
                return ParsedNode(ChoiceSequenceNode(token,nodes), False)
        return node

    
    """
    This method checks if a token any of the following operations: =, <, >, <=, >=, |, +, -
    Since all of this operations have the same priority and same values output, it is not needed to write them in different methods
    """
    def operation(self):
        # RULE --> op: term ((GT|LT|GE|LE|EQUAL|CHOICE|PLUS|MINUS) term)*

        left_node = self.term()

        # Checks if left node has been received and if the following token is one of the following tokens: : =, <, >, <=, >=, |, +, -

        if(left_node.hasSyntaxError == False and (self.check_type(self.current_token.type,
                [TokenTypes.GREATER,TokenTypes.GREATEREQ,TokenTypes.LOWER,TokenTypes.LOWEREQ, TokenTypes.PLUS,
                TokenTypes.MINUS]))):

                node = ParsedNode(None,True)
                
                # The while method "concatenates" the operations

                while(self.check_type(self.current_token.type,
                [TokenTypes.GREATER,TokenTypes.GREATEREQ,TokenTypes.LOWER,TokenTypes.LOWEREQ, TokenTypes.PLUS,
                TokenTypes.MINUS])):
                
                    token = self.current_token
                    self.forward()
                    right_node = self.term()
                    if(right_node.hasSyntaxError):
                        return right_node
                    
                    # Binds found operation to its left node
                    if(node.node == None):
                       node = ParsedNode(OperatorNode(token,left_node.node,right_node.node),False)
                    else: node = ParsedNode(OperatorNode(token,node.node,right_node.node),False)
                return node
        return left_node

    """
    Checks the same way in operation method but here it checks for *, /
    """
    def term(self) -> ParsedNode:
        # RULE --> factor ((MUL|DIV) factor)*
        
        left_node = self.factor() 

        if(left_node.hasSyntaxError == False and (self.check_type(self.current_token.type,[TokenTypes.MULTIPLY, TokenTypes.DIVIDE]))):
            node = ParsedNode(None,True)

             # The while method "concatenates" the operations
            while(self.check_type(self.current_token.type,[TokenTypes.MULTIPLY, TokenTypes.DIVIDE])):
               
                token = self.current_token
                self.forward()
                right_node = self.factor()
                if(right_node.hasSyntaxError):
                    return right_node
                
                # Binds found operation to its left node
                if(node.node == None):
                  node = ParsedNode(OperatorNode(token,left_node.node,right_node.node),False)
                else: node = ParsedNode(OperatorNode(token,node.node,right_node.node),False)
            return node
        return left_node
    
    """
    Checks for unary operations, Integers, brackets (highest priority)
    RULE -->  INTEGER  
        : brackets
        : (MINUS|PLUS) arith
        : func_call x() x
        : indexing     NOT IMPLEMENTING
        : --> means the same as (brackets|unary|func_call) just like in operation()
        only that for each if a different Node may be created not such as only OperationNode like in operation()
    """
    def factor(self) -> ParsedNode:
        token = self.current_token
        index = self.lexer.index

        #Integer check
        if(token.type == TokenTypes.INTEGER):
            self.forward()
            return ParsedNode(NumberNode(token),False)
        
        if(token.type == TokenTypes.FAIL):
            self.forward()
            return ParsedNode(FailNode(token),False)
        
        #Unary operation check
        if(self.check_type(self.current_token.type,[TokenTypes.PLUS, TokenTypes.MINUS])):
            self.forward()
            node = self.operation()
            if(node.hasSyntaxError):        # (--) --> Error needs (-- expr) or (--3)
                return ParsedNode(None, True)
            return ParsedNode(UnaryNode(token,node.node),False)
        
        
        node = self.sequence()
        
        #brackets check
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.brackets()
        
        #if node has failed check for loop
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.indexing()

        #if node has failed check for loop
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.scope()

         #if node has failed check for loop
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.binding()

        #if node has failed check indexing
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.func_call()
        
        #if node has failed check for loop
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.for_loop()

        #if node has failed check if statement
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.if_statement()
            
        if(node.hasSyntaxError):
            self.set_to_token(index,token)
            node = self.identifier()
        return node
    
    """
    Checks for brackets (highest priority)
    RULE --> brackets: LB expr RB
    """
    def brackets(self) -> ParsedNode: 
        if(self.current_token.type == TokenTypes.LBRACKET):
            node = self.expr()
        
            if(self.current_token.type == TokenTypes.RBRACKET):
                self.forward()
                return node
        return ParsedNode(None,True)
        


    """
    y := 8 y:=(x:int)  y:= method(...)...
    RULE --> scope BINDING expr
    """
    def binding(self) -> ParsedNode:
        left_node = self.identifier()

        if(left_node.hasSyntaxError == False):
            if(self.check_type(self.current_token.type,[TokenTypes.BINDING])):
                token = self.current_token
                self.forward()
                right_node = self.expr()
                if(right_node.hasSyntaxError == False):
                    return ParsedNode(BindingNode(token,left_node.node,right_node.node), False)
                else: return ParsedNode(None,True)
        return ParsedNode(None,True)

    """
    x:int
    RULE --> identifier COLON type 
    """
    def scope(self) -> ParsedNode:
        left_node = self.identifier()
        if(left_node.hasSyntaxError == False):
            if(self.check_type(self.current_token.type,[TokenTypes.COLON])):
                token = self.current_token
                self.forward()
                type = self.type()
                if(type.hasSyntaxError == False):
                    return ParsedNode(ScopeNode(token,[left_node.node],type.node),False) # Return Scope Node
                else: return ParsedNode(None,True)
        return ParsedNode(None,True)
    

    """
    variable/method name
    RULE --> identifier            NEED UPDATE
    """
    def identifier(self) -> ParsedNode:
        token = self.current_token
        if(token.type == TokenTypes.IDENTIFIER):
            self.forward()
            return ParsedNode(IdentifierNode(token), False)
        return ParsedNode(None, True) 
        
    """
    int or tuple(int,int) or array{int}
    RULE -->  INT                        
            : TUPLE LB type (,type)* RB 
    """
    def type(self) -> ParsedNode:
        # RULE -->  INT                        
        #        : TUPLE LB type (,type)* RB    

        token = self.current_token
        if(token.type == TokenTypes.INT_TYPE):
            self.forward()
            return ParsedNode(TypeNode(token),False)
        
        if(token.type == TokenTypes.TUPLE_TYPE):
            self.forward()
            if(self.current_token.type == TokenTypes.LBRACKET):
                self.forward()
                types:list[TypeNode] = []

                type = self.type()
                if(type.hasSyntaxError == False):
                    types.append(type.node)
                    if(self.check_type(self.current_token.type, [TokenTypes.COMMA])):
                        while(self.current_token.type == TokenTypes.COMMA):

                            self.forward()
                            t = self.type()

                            if(t.hasSyntaxError):  #If on error
                                return ParsedNode(None,True)
                            types.append(t.node) #else append to list of types
                     
            if(self.current_token.type == TokenTypes.RBRACKET):  
                self.forward()
                return ParsedNode(TypeNodeSequence(TokenTypes.TUPLE_TYPE,types), False)
            
        return ParsedNode(None, True) 
        
    """
    a[i:int]
    # RULE --> identifier SBL expr SBR
    """
    def indexing(self) -> ParsedNode:
        left_node = self.identifier()
        if(left_node.hasSyntaxError == False):
            if(self.current_token.type == TokenTypes.SBL):
                self.forward()
                expr_node = self.expr()
          
                if(expr_node.hasSyntaxError == False and self.current_token.type == TokenTypes.SBR):
                     self.forward()
                     return ParsedNode(IndexingNode(left_node.node.token,left_node.node,expr_node.node),False)
                return ParsedNode(None,True)
        return ParsedNode(None,True)
    
    """
    RUKE --> LB (expr COMMA expr)* RB                  --> tuple (n1,...)
             array CBL expr (COMMA expr)*? CBR         --> long-form syntax and singleton tuple/array array{n1} oder array{n1,...}
    """
    def sequence(self) -> ParsedNode:
        token = self.current_token

        nodes:list[BaseNode] = []

        # Tuple
        if(token.type == TokenTypes.LBRACKET):
            self.forward()
            node = self.expr()
            if(node.hasSyntaxError==False):
                nodes.append(node.node)
                while(self.current_token.type == TokenTypes.COMMA):
                    self.forward()
                    node = self.expr()
                    if(node.hasSyntaxError==False):
                        nodes.append(node.node)
                    else: return ParsedNode(None,True)
                if(len(nodes) > 1 and self.current_token.type == TokenTypes.RBRACKET):
                    self.forward()
                    return ParsedNode(SequenceNode(Token(TokenTypes.TUPLE_TYPE,TokenTypes.TUPLE_TYPE.value),nodes), False)


        # Array
        if(token.type == TokenTypes.ARRAY_TYPE):
                self.forward()
                if(self.current_token.type == TokenTypes.CBL):
                    self.forward()
                    node = self.expr()
                    if(node.hasSyntaxError == False):
                        nodes.append(node.node)
                        while(self.current_token.type == TokenTypes.COMMA):
                              self.forward()
                              node = self.expr()
                              if(node.hasSyntaxError==False):
                                nodes.append(node.node)
                              else: return ParsedNode(None,True)
                        if(self.current_token.type == TokenTypes.CBR):
                            self.forward()
                            return ParsedNode(SequenceNode(Token(TokenTypes.TUPLE_TYPE,TokenTypes.TUPLE_TYPE.value),nodes), False)
                        
        return ParsedNode(None, True)
                

    """
    Moves forward in the tokens list
    """
    def forward(self) -> None:
        print(self.current_token.__info__())
        self.lexer.forward()
        self.current_token = lexer.get_token(self.lexer.current_char)
        if self.current_token.type == TokenTypes.EOF:
            self.end = True
        

        
    """
    Checks if a type exists in the following types list
    """
    def check_type(self,type:TokenTypes,types:list[TokenTypes]) -> bool:
        return type in types
    

    """
    Sets current token back if a certain path lead to failure (Wrong syntax)
    May need it for later
    """
    def set_to_token(self,index, token): 
        self.current_token = token
        self.lexer.index = index

In [39]:
class Interpreter:
    def __init__(self, parser: Parser):
        self.parser = parser
        self.scopetable = ScopeTable()

    def interpret(self):
        tree = self.parser.parse()
        if tree != None:
            return self.visit(tree)
    
    def visit(self, node):
        if isinstance(node, ProgramNode):
                return self.visit_programNode(node)
        elif isinstance(node, BlockNode):
                return self.visit_blockNode(node)
        elif isinstance(node, ScopeNode):
                return self.visit_scopeNode(node)
        elif isinstance(node, BindingNode):
                return self.visit_bindingNode(node)
        elif isinstance(node, OperatorNode):
                return self.visit_operatorNode(node)
        elif isinstance(node, NumberNode):
                return self.visit_numberNode(node)
        elif isinstance(node, StatementNode):
                return self.visit_statementNode(node)
        elif isinstance(node, UnaryNode):
                return self.visit_unaryNode(node)
        elif isinstance(node, IdentifierNode):
                return self.visit_identifierNode(node)
        elif isinstance(node, TypeNode):
                return self.visit_typeNode(node)
        elif isinstance(node, TypeNodeSequence):
                return self.visit_typeNodeSequence(node)
        elif isinstance(node, ArgumentsNode):
                return self.visit_argumentsNode(node)
        elif isinstance(node, FuncCallNode):
                return self.visit_funcCallNode(node)
        elif isinstance(node, ParamsNode):
                return self.visit_paramsNode(node)
        elif isinstance(node, FuncDeclNode):
                return self.visit_funcDeclNode(node)
        elif isinstance(node, ForNode):
            return self.visit_forNode(node)
        elif isinstance(node, IfNode):
            return self.visit_ifNode(node)
        elif isinstance(node, RigidEqNode):
                return self.visit_rigidEqNode(node)
        elif isinstance(node, FlexibleEqNode):
                return self.visit_flexibleEqNode(node)
        elif isinstance(node, SequenceNode):
                return self.visit_sequenceNode(node)
        elif isinstance(node, ChoiceSequenceNode):
            return self.visit_choideSequenceNode(node)
        elif isinstance(node, IndexingNode):
                return self.visit_indexingNode(node)
        elif isinstance(node, ParsedNode):
                return self.visit(node.node)
        elif isinstance(node, FailNode):
                return self.visit_failNode(node)
    
    def visit_programNode(self, node: ProgramNode):
        return self.visit(node.node)

    def visit_blockNode(self, node: BlockNode):
        results = []
        for n in node.nodes:
            result = self.visit(n)
            if result != None:
                 results.append(result)
        
        return results

    def visit_scopeNode(self, node: ScopeNode):
        for n in node.nodes:
            self.scopetable.addScope(n.token.value, None, self.visit(node.type))

    def visit_bindingNode(self, node: BindingNode):
        self.scopetable.addScope(node.leftNode.token.value, node.rightNode, None)
        
    def visit_operatorNode(self, node: OperatorNode):
        val_left = self.visit(node.leftNode)
        val_right = self.visit(node.rightNode)

        # checks if it is a simple math operation.
        if type(val_left) == int and type(val_right) == int:
            return self.doOperation(val_left, val_right, node.token)

        if type(val_left) == BaseNode:
            if val_left.node.token.type == TokenTypes.FAIL:
                return val_left
        else:
             val_left = NumberNode(Token(TokenTypes.INTEGER, val_left))
        if type(val_right) == BaseNode:
            if val_right.node.token.type == TokenTypes.FAIL:
                    return val_right
        else:
             val_right = NumberNode(Token(TokenTypes.INTEGER, val_right))

        return self.GetNodeForOperation(val_left, val_right, node.token)

                
    def GetNodeForOperation(self,val_left,val_right, opToken:Token):
        nodes = []
        token = None

        if val_left.node.token.type == TokenTypes.CHOICE:
            token = val_left.node.token
            for n in val_left.node.nodes:
                if val_right.node.token.type == TokenTypes.CHOICE:
                    for n2 in val_right.node.nodes:
                       node = self.doOperation(n.value,n2.value,opToken)
                       nodes.append(node)
                else:nodes.append(self.doOperation(n.value,val_right.node.token.value,opToken)) 

        elif val_right.node.token.type == TokenTypes.CHOICE:
            token = val_right.node.token
            for n in val_right.node.nodes:
               nodes.append(self.doOperation(n.value,val_left.node.token.value,opToken)) 
               
        elif val_left.node.token.type == TokenTypes.INTEGER and val_right.node.token.type == TokenTypes.INTEGER:
             return self.doOperation(val_left.node.token.value,val_right.node.token.value,opToken)
        return SequenceNode(token, nodes)
                             
    def doOperation(self,val1:int,val2:int, token:Token):
        result = 0
        match token.type:
            case TokenTypes.DIVIDE:
                result = val1 // val2
            case TokenTypes.MULTIPLY:
                result = val1 * val2
            case TokenTypes.PLUS:              
                result = val1 + val2
            case TokenTypes.MINUS:
                result = val1 - val2
            case TokenTypes.DOT:
                return [i for i in range(val1, val2)]   

        if token.type == TokenTypes.EQUAL:
            if type(val1) == type(val2):
                if val1 == val2:
                    result = val1
        
        if token.type == TokenTypes.GREATER:
            if type(val1) == type(val2):
                if val1 > val2:
                    result = val1
        
        if token.type == TokenTypes.LOWER:
            if type(val1) == type(val2):
                if val1 < val2:
                    result = val1
            
        return  result 

    def visit_numberNode(self, node: NumberNode):
        return node.value
    
    def visit_statementNode(self, node: StatementNode):
        pass

    def visit_unaryNode(self, node: UnaryNode):
        pass

    def visit_identifierNode(self, node: IdentifierNode):
        # checks if the identifier already exists in the scopetable.
        for scope in self.scopetable.scopetable:
             if scope.symbol == node.token.value and scope.value != None:
                  return self.visit(scope.value)
        return node.token.value

    def visit_typeNode(self, node: TypeNode):
        return node.token.type

    def visit_typeNodeSequence(self, node: TypeNodeSequence):
        result = []
        for n in node.types:
            result.append(self.visit(n))
        return result

    def visit_argumentsNode(self, node: ArgumentsNode):
        pass

    def visit_funcCallNode(self, node: FuncCallNode):
        pass

    def visit_paramsNode(self, node: ParamsNode):
        pass

    def visit_funcDeclNode(self, node: FuncDeclNode):
        pass
    
    def visit_forNode(self, node: ForNode):
        if node.condition == None and node.expr == None and node.do == None:
            return self.visit(node.node)
        visitted_node = self.visit(node.node)
        
        if node.condition != None:
            visitted_condition = self.visit(node.condition)

        if node.expr != None:
            visitted_expr = self.visit(node.expr)
        
        if visitted_expr != None:
                return visitted_expr

    def visit_ifNode(self, node: IfNode):
        result_if = self.visit(node.if_node)
        if result_if != None:
            return self.visit(node.then_node)
        return self.visit(node.else_node)

    def visit_rigidEqNode(self, node: RigidEqNode):
        pass

    def visit_flexibleEqNode(self, node: FlexibleEqNode):
        leftResult = self.visit(node.left_node)
        self.scopetable.addScope(leftResult, node.right_node, None)
             

    def visit_sequenceNode(self, node: SequenceNode):
        # nodeStatus:NodeStatus = NodeStatus.VALUE_RECEIVABLE
        sequentor:Sequentor = Sequentor(node.nodes) 
        sequences = sequentor.getSequences()
        if len(sequences) == 0:
            return sequences[0]
        return ChoiceSequenceNode(Token(TokenTypes.CHOICE,sequences))

    def visit_indexingNode(self, node: IndexingNode):
        for scope in self.scopetable.scopetable:
            if node.identifier.token.value == scope.symbol:
                value = self.visit(scope.value)
                if node.index.value >= len(value):
                    print("Exception -> Index out of range")
                    return

                return value[node.index.value]
    
    def visit_choideSequenceNode(self,node):
        nodes = []

        # Choice appends all of its sequence, not containing false?
        if node.token.type == TokenTypes.CHOICE:
            for n in node.nodes:
                    current_n = self.visit(n)

                    # Do on not assigned value
                    if current_n.type == NodeStatus.NOT_ASSIGNED_YET:
                         nodeStatus = NodeStatus.NOT_ASSIGNED_YET

                    # Do on error
                    elif current_n.type == NodeStatus.ERROR:
                         nodeStatus = NodeStatus.ERROR
                         print("Error in sequence visitor for CHOICE")
                         break 
                    
                    # Skip fail node
                    if current_n.node.token.type != TokenTypes.FAIL:
                         nodes.append(current_n.node)  

            # If choise sequence is empty, return false?
            if(len(nodes) == 0):
                return VisitorNode(nodeStatus,FailNode(Token(TokenTypes.FAIL,TokenTypes.FAIL.value)))
            
            # If choise has atleast one return the node it only contains instead od a choice sequence node.
            if(len(nodes) == 1):
                 return VisitorNode(nodeStatus,nodes[0].node) 
            
            # At last, return choice sequence node wit all visited values.
            return VisitorNode(nodeStatus,ChoiceSequenceNode(node.token, nodes))

    def visit_failNode(self, node: FailNode):
        print("Fail")


In [40]:
text = "x:=(y|2); y:=(7|8); (x,y)"
# text = "x:int; x=7; if(x<20) then x else 333"
lexer = lexicon(text)
parser = Parser(lexer)
interpreter = Interpreter(parser)
result = interpreter.interpret()
result

TokenTypes.IDENTIFIER: x
TokenTypes.IDENTIFIER: x
TokenTypes.IDENTIFIER: x
TokenTypes.IDENTIFIER: x
TokenTypes.IDENTIFIER: x
TokenTypes.BINDING: :=
TokenTypes.LBRACKET: (
TokenTypes.IDENTIFIER: y
TokenTypes.IDENTIFIER: y
TokenTypes.IDENTIFIER: y
TokenTypes.IDENTIFIER: y
TokenTypes.IDENTIFIER: y
TokenTypes.CHOICE: |
TokenTypes.INTEGER: 2
TokenTypes.LBRACKET: (
TokenTypes.IDENTIFIER: y
TokenTypes.IDENTIFIER: y
TokenTypes.IDENTIFIER: y
TokenTypes.IDENTIFIER: y
TokenTypes.IDENTIFIER: y
TokenTypes.CHOICE: |
TokenTypes.INTEGER: 2
TokenTypes.LBRACKET: (
TokenTypes.IDENTIFIER: y
TokenTypes.IDENTIFIER: y
TokenTypes.IDENTIFIER: y
TokenTypes.IDENTIFIER: y
TokenTypes.IDENTIFIER: y
TokenTypes.CHOICE: |
TokenTypes.INTEGER: 2
TokenTypes.LBRACKET: (
TokenTypes.IDENTIFIER: y
TokenTypes.IDENTIFIER: y
TokenTypes.IDENTIFIER: y
TokenTypes.IDENTIFIER: y
TokenTypes.IDENTIFIER: y
TokenTypes.CHOICE: |
TokenTypes.INTEGER: 2
TokenTypes.LBRACKET: (
TokenTypes.IDENTIFIER: y
TokenTypes.IDENTIFIER: y
TokenTypes.IDEN

RecursionError: maximum recursion depth exceeded while getting the str of an object

## Testing for statement

In [ ]:
text = "for{i:int; i=3; i<7}"
# text = "x,y:int; x=7; y=3; x+y"
lexer = lexicon(text)
parser = Parser(lexer)
interpreter = Interpreter(parser)
result = interpreter.interpret()
print("\n Result = " + str(result))

TokenTypes.FOR: for
TokenTypes.CBL: {
TokenTypes.IDENTIFIER: i
TokenTypes.COLON: :
TokenTypes.INT_TYPE: int
TokenTypes.SEMICOLON: ;
TokenTypes.IDENTIFIER: i
TokenTypes.EQUAL: =
TokenTypes.INTEGER: 3
TokenTypes.SEMICOLON: ;
TokenTypes.IDENTIFIER: i
TokenTypes.IDENTIFIER: i
TokenTypes.IDENTIFIER: i
TokenTypes.IDENTIFIER: i
TokenTypes.IDENTIFIER: i
TokenTypes.IDENTIFIER: i
TokenTypes.LOWER: <
TokenTypes.INTEGER: 7
TokenTypes.CBR: }

 Result = 3


# UNIFIKATION

TypeError: 'SequenceNode' object is not iterable